In [10]:
import pandas as pd 
import plotly.express as px
import numpy as np
dfo = pd.read_csv("..\\Data\\SmA-Four-Tank-Batch-Process_V2.csv",sep=';',parse_dates=['timestamp'])

In [11]:
df=dfo.copy()
df=df[df['DeviationID ValueY']!=0]
df['Dev']=df['DeviationID ValueY']!=1
df['Dev']=df['Dev'].astype('int')
df['Dev']=df['Dev'].astype('str')
df_clean=df.copy()
df_clean = df_clean[df_clean["CuStepNo ValueY"] != 2]

In [12]:
map = {1 : 1, 7 : 2, 8 : 3, 3:4}
df_clean["Step_new"] = df_clean["CuStepNo ValueY"].map(map)
df_clean["Step_new"] = df_clean["Step_new"].astype(int)

df_clean["Step_shift"] = df_clean["Step_new"].shift(1)
df_clean['Step_shift']=df_clean['Step_shift'].fillna(0)
df_clean['Step_shift']=df_clean['Step_shift'].astype(int)

In [13]:
index=0
def id(x):
    global index 
    #print(index,x['Step_new'],x['Step_shift'])
    if (x['Step_new']==1) & (x['Step_shift']==4):
        index+=1
        return index
    else:
        return index

In [14]:
df_clean['id']=df_clean.apply(id,axis=1)

In [16]:
df_min=df_clean.groupby('id').aggregate({'timestamp':'min'}).reset_index()
df_min.columns=['id','t_min']
df_with_min=pd.merge(df_clean,df_min,on='id')
df_with_min['div_min']=df_with_min['timestamp']-df_with_min['t_min']

In [26]:
df_with_min.groupby(['div_min','Dev'])[['Step_new']].sum()

Step_new
div_min         Dev          
0 days 00:00:00 0          35
                1         227
0 days 00:00:01 0          35
                1         227
0 days 00:00:02 0          35
...                       ...
0 days 00:33:12 0           4
0 days 00:33:13 0           4
0 days 00:33:14 0           4
0 days 00:33:15 0           4
0 days 00:33:16 0           4

[3841 rows x 1 columns]

In [ ]:
#px.scatter(df_with_minmax[df_with_minmax['Dev']=='1'],x='div_min',y='Dev',marginal_x="violin")